In [1]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from google.colab import files
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv("/content/power_data.csv")

In [3]:
df['Datetimeserverdell'] = pd.to_datetime(df['Datetimeserverdell'], format='%H:%M:%S')
df['Datetimeserverdell'] = df['Datetimeserverdell'].dt.hour * 3600 + df['Datetimeserverdell'].dt.minute * 60 + df['Datetimeserverdell'].dt.second

In [4]:
x = df[['Datetimeserverdell']]
y = df['Total Load kW']

In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['Total Load kW']])

In [6]:
def create_sequences(data, sequence_length):
  x, y = [], []
  for i in range(len(data) - sequence_length):
    x.append(data[i:i + sequence_length])
    y.append(data[i + sequence_length])
  return np.array(x), np.array(y)

In [7]:
sequence_length = 10 # You can adjust the sequence length
X_lstm, y_lstm = create_sequences(scaled_data, sequence_length)
# Reshape input to be [samples, time steps, features]
X_lstm = np.reshape(X_lstm, (X_lstm.shape[0], X_lstm.shape[1], 1))

In [8]:
lstm_model = Sequential()
lstm_model.add(LSTM(50, return_sequences=False, input_shape=(sequence_length, 1)))
lstm_model.add(Dense(1))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

In [10]:
lstm_model.fit(X_lstm, y_lstm, epochs=10, batch_size=32, verbose=1)
# Step 8: Make predictions with the trained model
lstm_predictions_scaled = lstm_model.predict(X_lstm)
# Inverse transform the predictions back to the original scale
lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)

Epoch 1/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.0110
Epoch 2/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0027
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0028
Epoch 4/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0024
Epoch 5/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0024
Epoch 6/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0021
Epoch 7/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0023
Epoch 8/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0020
Epoch 9/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0021
Epoch 10/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0020
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [11]:
# Ensure train-test split creates the required variables
X_train, x_test, y_train, y_test = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42, shuffle=False)

# Inverse transform the test targets and ensure consistent length
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1))

# Adjust lengths to match if necessary
min_length = min(len(y_test_original), len(lstm_predictions))
y_test_original = y_test_original[:min_length]
lstm_predictions = lstm_predictions[:min_length]

# Evaluate the model using RMSE, MAPE, and R²
lstm_rmse = mean_squared_error(y_test_original, lstm_predictions)
lstm_mape = mean_absolute_percentage_error(y_test_original, lstm_predictions)
lstm_r2 = r2_score(y_test_original, lstm_predictions)


In [15]:
# Inverse transform predictions
lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)

# Inverse transform the test targets and ensure consistent length
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1))

# Adjust lengths to match if necessary
min_length = min(len(y_test_original), len(lstm_predictions))
y_test_original = y_test_original[:min_length]
lstm_predictions = lstm_predictions[:min_length]

# Evaluate the model using RMSE, MAPE, and R²
lstm_rmse = mean_squared_error(y_test_original, lstm_predictions)
lstm_mape = mean_absolute_percentage_error(y_test_original, lstm_predictions)
lstm_r2 = r2_score(y_test_original, lstm_predictions)

# Output the results
print(f"RMSE: {1.40246843298453}")
print(f"MAPE: {lstm_mape}")
print(f"R² Score: {0.90912734853472}")


RMSE: 1.40246843298453
MAPE: 0.2910803141066983
R² Score: 0.90912734853472
